In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/house-prices-advanced-regression-techniques/sample_submission.csv
/kaggle/input/house-prices-advanced-regression-techniques/data_description.txt
/kaggle/input/house-prices-advanced-regression-techniques/train.csv
/kaggle/input/house-prices-advanced-regression-techniques/test.csv


In [2]:
import pandas as pd 
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import random
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import OrdinalEncoder
from sklearn.cluster import DBSCAN
from sklearn.ensemble import RandomForestClassifier
from sklearn.feature_selection import mutual_info_regression
import warnings

warnings.filterwarnings('ignore')


def load_csv(file_path, index_col='Id', parse_dates=True):
    df = pd.read_csv(file_path,  parse_dates=parse_dates, index_col='Id')
    return df

def delete_noise(X, indices):
    X.drop(indices, inplace = True, errors='ignore')
    X.reset_index(drop=True)    
    return

def get_irrilevant_cols(X, perc_treshold):
    row_count = X.shape[0]
    cols_to_delete = []
    for col in X.columns:
        values_counting = X[col].value_counts().reset_index().rename(columns={'index': 'val', col: 'counts'})
        top_values = values_counting.sort_values(by='count', ascending=False).head(2)
        perc_top= (top_values.iloc[0,1]/row_count)*100
        if(perc_top>perc_treshold):
            #print(f'percentage top:{perc_top}')
            #print(f"Top values for column '{col}'. Not nan values:{X[col].count()}:")
            #print(top_values)
            cols_to_delete.append(col)

    return cols_to_delete

def find_outliers(X, col, quantile_param):
    if X[col].dtype in ['int64', 'float64']:  # Se la colonna è numerica
        Q1 = X[col].quantile(0.25)
        Q3 = X[col].quantile(0.75)
        IQR = Q3 - Q1
        
        lower_bound = Q1 - 1.5 * IQR
        upper_bound = Q3 + 1.5 * IQR
        if lower_bound<0:
            lower_bound=0
        print(f'lower and upper bound for {col}: {lower_bound} : {upper_bound}')
        outlier_indices = X[(X[col] < lower_bound) | (X[col] > upper_bound)].index
    
    elif X[col].dtype == 'object':  # Se la colonna è categoriale
        # Ottieni le frequenze delle categorie
        freq = X[col].value_counts(normalize=True)
        # Identifica le categorie che compaiono meno del 5% delle volte
        rare_categories = freq[freq < 0.05].index
        print(f'rare_categories for {col}: {rare_categories}')
        
        outlier_indices = X[X[col].isin(rare_categories)].index
    
    else:
        raise ValueError("Tipo di dati non supportato per la colonna: " + str(X[col].dtype))
    
    return outlier_indices.to_numpy()

def delete_outliers(X, outliers):
    for key in outliers:
        X.drop(key, inplace = True, errors='ignore')

def delete_outliers(X, outliers):
    for key in outliers:
        X.drop(key, inplace = True, errors='ignore')

    return X

def label_encode(X):
    le = LabelEncoder()
    return np.array([le.fit_transform(col) for col in X.T]).T  

def to_lowercase(df):
    return df.applymap(lambda x: x.lower() if isinstance(x, str) else x)
    

def encode_with_custom_order(df, column_name, order_values):
    unique_values = [value.lower() if isinstance(value, str) else value for value in df[column_name].unique()]
    # Filtra solo i valori che sono nella custom_order
    valid_values = [value.lower() for value in order_values if value in unique_values]
    return valid_values

def remove_original_columns(X):
    return X[:, -3:]  # Supponendo che l'output di svd abbia 3 colonne


def coordinates(neighborhood):
    coordinates={}
    coordinates['Blmngtn']= [42.0220 , 93.6170]
    coordinates['Blueste']=[42.0145 , 93.6130]
    coordinates['BrDale']= [42.0188 , 93.6002 ]
    coordinates['BrkSide']=[42.0213 , 93.6178]
    coordinates['ClearCr']= [42.0240 , 93.6230] 
    coordinates['CollgCr']= [42.0315 , 93.6157] 
    coordinates['Crawfor']= [42.0236 , 93.6229]
    coordinates['Edwards']= [42.0212 , 93.6095] 
    coordinates['Gilbert']= [42.0420 , 93.6378]
    coordinates['IDOTRR']=[42.0330 , 93.5900] 
    coordinates['MeadowV']= [42.0140 , 93.6155]
    coordinates['Mitchel']= [42.0332 , 93.6053] 
    coordinates['Names']= [42.0270 , 93.6145] 
    coordinates['NoRidge']=[42.0290 , 93.5988]
    coordinates['NPkVill']= [42.0278 , 93.5867]
    coordinates['NridgHt']=[42.0255 , 93.5935 ]
    coordinates['NWAmes']= [42.0312 , 93.6293]
    coordinates['OldTown']=[42.0293 , 93.6171]
    coordinates['SWISU']= [42.0214 , 93.6178] 
    coordinates['Sawyer']= [42.0080, 93.6290 ]
    coordinates['SawyerW']=[42.0065 , 93.6280]
    coordinates['Somerst']= [42.0240 , 93.6050] 
    coordinates['StoneBr']= [42.0290 , 93.6160 ]
    coordinates['Timber']= [42.0350 , 93.5990] 
    coordinates['Veenker']= [42.0358 , 93.6340]
    
    return coordinates.get(neighborhood, "[0,0]")

**IRRILEVANT COLS AND OUTLIERS**

In [3]:
X = load_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv", index_col='Id')

# Escludiamo tutti i campi che non possono essere indicativi
irrilevant_cols = get_irrilevant_cols(X, perc_treshold=86)
#print(irrilevant_cols)
fields_ok = [col for col in X.columns if col not in irrilevant_cols]

outliers_indexes = {}
outliers_for_feature = {}
quantile_tweak = 1.5

fields_to_exclude = []
fields_to_exclude.append('MSSubClass')
fields_to_exclude.append('Neighborhood')
fields_to_exclude.append('YearRemodAdd')
fields_to_exclude.append('YrSold')
fields_to_exclude.append('YearBuilt')
for col in X.columns:
    if col not in fields_to_exclude:
        outliers_for_feature = find_outliers(X, col, quantile_tweak)
        for value in outliers_for_feature:
            if value in outliers_indexes:
                outliers_indexes[value].append(col)
            else:
                outliers_indexes[value] = [col]

outliers_size={}
for key, value in outliers_indexes.items():
    outliers_size[key] = len(value)
    
outliers_df = pd.DataFrame(list(outliers_size.items()), columns=['Chiave', 'Lunghezza'])
outliers_df.sort_values(by='Lunghezza', ascending=False)


X.reset_index(drop=True)
#outliers_df[outliers_df['Lunghezza']>8].sort_values(by='Lunghezza', ascending=False)['Chiave'].to_list()

rare_categories for MSZoning: Index(['FV', 'RH', 'C (all)'], dtype='object', name='MSZoning')
lower and upper bound for LotFrontage: 27.5 : 111.5
lower and upper bound for LotArea: 1481.5 : 17673.5
rare_categories for Street: Index(['Grvl'], dtype='object', name='Street')
rare_categories for Alley: Index([], dtype='object', name='Alley')
rare_categories for LotShape: Index(['IR2', 'IR3'], dtype='object', name='LotShape')
rare_categories for LandContour: Index(['Bnk', 'HLS', 'Low'], dtype='object', name='LandContour')
rare_categories for Utilities: Index(['NoSeWa'], dtype='object', name='Utilities')
rare_categories for LotConfig: Index(['FR2', 'FR3'], dtype='object', name='LotConfig')
rare_categories for LandSlope: Index(['Mod', 'Sev'], dtype='object', name='LandSlope')
rare_categories for Condition1: Index(['Artery', 'RRAn', 'PosN', 'RRAe', 'PosA', 'RRNn', 'RRNe'], dtype='object', name='Condition1')
rare_categories for Condition2: Index(['Feedr', 'Artery', 'RRNn', 'PosN', 'PosA', 'RRAn

,MSSubClass,MSZoning,LotFrontage,LotArea,Street,Alley,LotShape,LandContour,Utilities,LotConfig,...,PoolArea,PoolQC,Fence,MiscFeature,MiscVal,MoSold,YrSold,SaleType,SaleCondition,SalePrice
0,60,RL,65.0,8450,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,2,2008,WD,Normal,208500
1,20,RL,80.0,9600,Pave,NaN,Reg,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,5,2007,WD,Normal,181500
2,60,RL,68.0,11250,Pave,NaN,IR1,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,9,2008,WD,Normal,223500
3,70,RL,60.0,9550,Pave,NaN,IR1,Lvl,AllPub,Corner,...,0,NaN,NaN,NaN,0,2,2006,WD,Abnorml,140000
4,60,RL,84.0,14260,Pave,NaN,IR1,Lvl,AllPub,FR2,...,0,NaN,NaN,NaN,0,12,2008,WD,Normal,250000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1455,60,RL,62.0,7917,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,8,2007,WD,Normal,175000
1456,20,RL,85.0,13175,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,MnPrv,NaN,0,2,2010,WD,Normal,210000
1457,70,RL,66.0,9042,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,GdPrv,Shed,2500,5,2010,WD,Normal,266500
1458,20,RL,68.0,9717,Pave,NaN,Reg,Lvl,AllPub,Inside,...,0,NaN,NaN,NaN,0,4,2010,WD,Normal,142125


In [4]:
import xgboost as xgb
from sklearn.ensemble import RandomForestRegressor
from sklearn.tree import DecisionTreeRegressor
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.linear_model import Lasso
from sklearn.model_selection import cross_val_score
from sklearn.feature_selection import mutual_info_regression
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder, OrdinalEncoder, FunctionTransformer
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.decomposition import PCA
from sklearn.decomposition import TruncatedSVD
from sklearn.preprocessing import FunctionTransformer


X = load_csv("/kaggle/input/house-prices-advanced-regression-techniques/train.csv", index_col='Id')
######################################################################################

##  OCCORRE TRASFORMARE IN STRINGA TUTTI I CAMPI CHE DI FATTO SONO CATEGORIE MA NON HANNO CONCETTO DI ORDINAMENTO
##  OPPURE DI DISTANZA

## TUTTE LE FEATURES CHE RAPPRESENTANO UN ORDINE VANNO TRASFORMATE IN ORDINAL ENCODING

## ANNO COSTRUZIONE e RISTRUTTURAZIONE DI PER SE' VOGLIONO DIRE POCO SE NON E' COMBINATO CON ANNO VENDITA, andrebbero quindi sottratti
## QUESTO VALE PER TUTTI GLI ANNI


## CREARE VARIABILI QUALITA' QUANTITA' --> normalizzare prima di moltiplicare oppure creare qualità proporzionale
## Qualità/numero elementi
## LO STESSO VALE Per Tipologia*area nel momento in cui la tipologia sia ordinabile per qualità

## Qualità e condizioni può essere considerata come media ponderata qualitàtotale = w * qualità + (1-w) condizioni
## Oppure semplicemente moltiplicandole

# PRIMA DI DEFINIRE UNA COLONNA IRRILEVANTE OCCORRE CAPIRE SE HA RELAZIONI CON ALTRE FEATURES
# irrilevant_cols:['Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 'RoofMatl', 
# 'ExterCond', 'BsmtCond', 'BsmtFinType2', 'BsmtFinSF2', 'Heating', 'CentralAir', 'Electrical', 'LowQualFinSF', 
# 'BsmtHalfBath', 'KitchenAbvGr', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', '3SsnPorch', 'ScreenPorch', 
# 'PoolArea', 'MiscVal', 'SaleType']


######################################################################################
features_engeneered = ['MSSubClass','YrSold','YearBuilt','YearRemodAdd','GarageYrBlt','OverallQual','OverallCond',
                      '1stFlrSF','2ndFlrSF','BsmtUnfSF','GarageArea'] #,'LowQualFinSF'

def features_engeneering(X):
    del_fields = []
    X['MSSubClass'] = X['MSSubClass'].astype('object')
    X['HouseAge'] = X['YrSold']-X['YearBuilt']
    X['RemodeAge']= X['YrSold']-X['YearRemodAdd']
    X['GarageAge'] = X['YrSold']-X['GarageYrBlt']
    del_fields.append('YrSold')
    del_fields.append('YearBuilt')
    del_fields.append('YearRemodAdd')
    del_fields.append('GarageYrBlt')

    weightQual = 0.6
    X['HouseQualCond'] = weightQual * X['OverallQual'] + (1-weightQual) * X['OverallCond']
    del_fields.append('OverallQual')
    del_fields.append('OverallCond')
    #X['ExterQualCond'] = weightQual* X['ExterQual']+ (1-weightQual)X['ExterCond']
    del_fields.append('1stFlrSF')
    del_fields.append('2ndFlrSF')
    #X['LowQualFinSF'] = (-1) * X['LowQualFinSF']
    X['BsmtUnfSF'] = (-1) * X['BsmtUnfSF']
    # GarageArea è pleonastico
    del_fields.append('GarageArea')
    
    X = X.drop(columns=del_fields)
    #print(f'after features_engeneering: {X.columns.tolist()}')
    return X


irrilevant_cols = get_irrilevant_cols(X, perc_treshold=86)
X = X.drop(columns=irrilevant_cols)
print(f'irrilevant_cols:{irrilevant_cols}')


#X = X[fields_ok]

# Pulizia df: noise, irrilevant cols, outliers 
# Delete noise 524, 1299 
# 198, 636, 1032, 1174
#noise = outliers_df[outliers_df['Lunghezza']>9].sort_values(by='Lunghezza', ascending=False)['Chiave'].to_list()
#print(noise)
#delete_noise(X, [1299, 524, 770, 636])#, 198, 314, 186, 247, 637]) #--> sono totalmente ingiustificati
#X.reset_index(drop=True)

y = X.pop('SalePrice')
features_engeneering(X)
# Seleziona le feature che desideri utilizzare (assicurati che questi siano i nomi corretti)
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1)

X_train = X_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)

numeric_features = [item for item in X.columns if X[item].dtype in('int64','float64')]
all_category_features = [item for item in X.columns if item not in numeric_features]

# Pipeline per label_engeneering features_engeneering
feature_creator = Pipeline(steps=[
    ('feature_creator', FunctionTransformer(features_engeneering, validate=False))
])

# Pipeline per la codifica One-Hot
#onehot_transformer = Pipeline(steps=[
#    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
#    ('onehot', OneHotEncoder(handle_unknown='ignore')),
#])


# Pipeline per la codifica delle etichette
label_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='constant', fill_value='NA')),
    ('cat_label', FunctionTransformer(label_encode, validate=False))
])


transformers=[]
#untransformed_features = ['price_mean_nbrhd','cluster_GarageQualCond','cluster_Sale','cluster_Exterior','cluster_Lot']# Sono le feature di target encoding ed i cluster per esempio

#transformers.append(('onehot', onehot_transformer, features_OneHot))
transformers.append(('feature_eng', feature_creator, features_engeneered))  
transformers.append(('label', label_transformer, all_category_features))
transformers.append(('num', Pipeline(steps=[('imputer', SimpleImputer(strategy='mean')),
                                            ('scaler', StandardScaler())]), numeric_features))
#transformers.append(('pca_Porch', Pipeline([
#            ('imputer', SimpleImputer(strategy='constant', fill_value=0)),
#            ('scaler', StandardScaler()),
#            ('pca', PCA(n_components=4)),
#            ('remove_columns', FunctionTransformer(remove_original_columns, validate=False))  
#]), pca_Porch))
#transformers.append(('passthrough', 'passthrough', untransformed_features))  # Passa le feature non trattate

preprocessor = ColumnTransformer(transformers=transformers)  

# Verifico la struttura dati
X_train_encoded = preprocessor.fit_transform(X_train, y_train)


# Clustering
#clustering_Lot = Clustering(features_to_cluster=cluster_Lot, n_clusters=8, cluster_name='cluster_Lot', fill_value='NA') 

model = XGBRegressor(objective='reg:squarederror')

# Pipeline finale
pipeline = Pipeline(steps=[
#    ('target_encoder', TargetEncoder(weight=0.7)),
#    ('clustering_gar', clustering_Garage),  
    ('preprocessor', preprocessor),
    ('model', model)
])


## RANDOM SEARCH
#param_dist = {
#    'model__n_estimators': np.arange(100, 150, 10),
#    'model__max_depth': np.arange(10, 20),
#    'model__min_samples_split': np.arange(2, 10),
#    'model__min_samples_leaf': np.arange(1, 5)
#}

# Creare l'oggetto RandomizedSearchCV
#random_search = RandomizedSearchCV(estimator=pipeline, param_distributions=param_dist,
#                                   scoring='neg_mean_squared_error', n_iter=100,
#                                   cv=5, n_jobs=-1, verbose=2, random_state=42)

# Eseguire la ricerca
#random_search.fit(X_train, y_train)

# Mostrare i migliori parametri
#print("Migliori parametri:", random_search.best_params_)
#print("Miglior punteggio:", random_search.best_score_)

# Cross validation
# Multiply by -1 since sklearn calculates *negative* MAE
#scores = -1 * cross_val_score(pipeline, X, y,
#                              cv=5,
#                              scoring='neg_mean_absolute_error')

#print("MAE scores:\n", scores)

# SENZA CROSS
# Fit e previsione


pipeline.fit(X_train, y_train)

predictions = pipeline.predict(X_test)

# Calcolo della MAE
mae = mean_absolute_error(y_test, predictions)
print(f'Mean Absolute Error (MAE): {mae}')


#X_train_encoded = encoder.fit_transform(X_train, y_train)

# Aggiungi il nuovo campo di encoding anche ai dati di test e inferenza
#X_test_encoded = encoder.transform(X_test)
#validation_data = pd.DataFrame({'Neighborhood': ['A', 'E']})  # 'E' è sconosciuto
#validation_encoded = encoder.transform(validation_data)

#inferenced_data = pd.DataFrame({'Neighborhood': ['A', 'E']})
#inferenced_encoded = encoder.transform(inferenced_data)
# [17989.16509925 17635.3600564  17921.45573071 16318.42176247 19576.79822101] BASE
# [16505.81516935 18995.93069023 17495.46746233 15562.60597304 17666.75046929]
# [16379.85285748 18059.44012411 18324.48427671 15149.93785626 18401.57595722]
# After Outliers handling
# [13766.04136004 12643.31778706 13143.98610186 13504.49152035 14066.56898189]
# After feature selection

# BEST With all fields Mean Absolute Error (MAE): 18731.190362799658
# escludendo colonne irrilevanti al 86 come threshold :
# Mean Absolute Error (MAE): 16844.136549300798


# Escludendo rumore (9)
# Mean Absolute Error (MAE): 16431.29962550172
# Escludendo [1299, 524, 770, 636]
# Mean Absolute Error (MAE): 16106.716756650458

irrilevant_cols:['Street', 'LandContour', 'Utilities', 'LandSlope', 'Condition1', 'Condition2', 'RoofMatl', 'ExterCond', 'BsmtCond', 'BsmtFinType2', 'BsmtFinSF2', 'Heating', 'CentralAir', 'Electrical', 'LowQualFinSF', 'BsmtHalfBath', 'KitchenAbvGr', 'Functional', 'GarageQual', 'GarageCond', 'PavedDrive', '3SsnPorch', 'ScreenPorch', 'PoolArea', 'MiscVal', 'SaleType']
Mean Absolute Error (MAE): 16844.136549300798


In [5]:
X_train_encoded.shape

(1022, 63)